In [18]:
!pip install bertopic
!pip install bertopic[visualization]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic

In [20]:
import os
import os
import pandas as pd
df = pd.read_csv(r"/kaggle/input/kcctn-final-2/train_TamilNadu.csv")


In [21]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
df = df.dropna(subset=['QueryText_processed'])
df['paper_text_processed'] = df['QueryText_processed'].map(lambda x: re.sub('[,\.!?]', '', x).lower())
df['paper_text_processed'].head()

import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.paper_text_processed.values.tolist()

In [22]:
data = df.paper_text_processed.values.tolist()
documents=data
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)

import gensim.corpora as corpora
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1)]


In [25]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import IncrementalPCA
from bertopic.vectorizers import OnlineCountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

cluster_model = MiniBatchKMeans(n_clusters=7, random_state=0)
vectorizer_model = OnlineCountVectorizer(stop_words="english")
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

topic_model = BERTopic(
    hdbscan_model=cluster_model, 
    vectorizer_model=vectorizer_model, 
    ctfidf_model=ctfidf_model,
)

topics, probabilities = topic_model.fit_transform(data)


KeyboardInterrupt: 

In [ ]:
import pandas as pd

topics = topic_model.get_topics()

topics_data = []
for topic_num, topic_keywords in topics.items():
    keywords_str = ', '.join([word for word, score in topic_keywords])
    topics_data.append({'Topic': topic_num, 'Keywords': keywords_str})

# Create the DataFrame using the list of dictionaries
topics_dataframe = pd.DataFrame(topics_data)

print(topics_dataframe)

In [ ]:
topics_dataframe['Keywords']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel

topics = topics_dataframe['Keywords'].apply(lambda x: x.split(', ')).tolist()
# print(topics)
# Calculate coherence
cm = CoherenceModel(topics=topics, texts=data_words, dictionary=id2word, coherence='u_mass')
coherence_per_topic = cm.get_coherence()

print("Coherence per Topic:", coherence_per_topic)


In [ ]:
topic_model.get_topic_freq()

In [ ]:
model.get_topic(10)

In [ ]:
topics_per_class = model.topics_per_class(data)
model.visualize_topics_per_class(topics_per_class)

In [ ]:
x=model.topics_
print(x)

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_heatmap()